### a) Construcción de Dataframe

In [1]:
import pandas as pd
df = pd.read_csv('text_emotion.csv')
print df.head()

print list(df.columns.values)
registros,atributos = df.shape
print "Cantidad de registros",registros
print "Cantidad de atributos",atributos


     tweet_id   sentiment       author  \
0  1956967341       empty   xoshayzers   
1  1956967666     sadness    wannamama   
2  1956967696     sadness    coolfunky   
3  1956967789  enthusiasm  czareaquino   
4  1956968416     neutral    xkilljoyx   

                                             content  
0  @tiffanylue i know  i was listenin to bad habi...  
1  Layin n bed with a headache  ughhhh...waitin o...  
2                Funeral ceremony...gloomy friday...  
3               wants to hang out with friends SOON!  
4  @dannycastillo We want to trade with someone w...  
['tweet_id', 'sentiment', 'author', 'content']
Cantidad de registros 40000
Cantidad de atributos 4


Este dataset consta de 40000 tweets, cada tweet tiene una id asociada,  una emoción mutuamente excluyente con las demás, el autor y el texto.

In [2]:
def clase_ocurrencia(df,campo):
    clases = {}
    for index,row in df.iterrows():
        if row[campo] not in clases.keys():
                clases[row[campo]] = 1
        else:
            clases[row[campo]] += 1
    return clases

emotions = clase_ocurrencia(df,'sentiment')
print "Clases y la cantidad de registros por clases"
print emotions




Clases y la cantidad de registros por clases
{'love': 3842, 'relief': 1526, 'neutral': 8638, 'anger': 110, 'sadness': 5165, 'empty': 827, 'surprise': 2187, 'fun': 1776, 'enthusiasm': 759, 'happiness': 5209, 'hate': 1323, 'worry': 8459, 'boredom': 179}


### b) Construcción de conjuntos

In [3]:
import numpy as np
msk = np.random.rand(len(df)) < 0.8
print msk
df_train = df[msk]
df_test = df[~msk]
print(len(df_train))
print(len(df_test))

[False False  True ..., False  True  True]
31987
8013


### c) Pre procesamiento 

Para aumentar la eficacia de las caracterı́sticas extraı́das es conveniente ejecutar algunas técnicas de pre-
procesamiento básicas como: pasar todo el texto a minúsculas (lower-casing), eliminar signos de puntuación
y eliminar palabras sin significado como artı́culos, pronombres y preposiciones (stop word removal [9]). Otra
técnica que suele ser útil para obtener buenas caracterı́sticas (features) es la lematización [11], es decir la
reducción de todas las palabras a su tronco léxico base. Una técnica similar y más utilizada en la práctica es
el stemming [10]. Varias de éstas están implementadas en la libreria nltk [13] para python.

In [4]:
tweets = []
for index,row in df.iterrows():
    tweets.append(row['content'])
    
from nltk.tokenize import TweetTokenizer

tknzr = TweetTokenizer()
tweet_index = 0
for i in tweets:
    tweets[tweet_index] = tknzr.tokenize(i)
    tweet_index += 1
    
# tweets => lista con tweets tokenizados

    


In [17]:
import re
stoplist = stopwords.words('english')
tweet_index = 0
for tweet in tweets:
    token_sin_stopwords = [ token for token in tweet  if token not in stoplist and len(token) != 1] 
    token_sin_stopwords = [ re.sub('([\W_]+)', '', token) for token in token_sin_stopwords ]  
    tweets[tweet_index] = token_sin_stopwords
    tweet_index += 1
    



### d ) Criterio para un problema binario

definir:neutral y empty, surprise 
+1 => love, relief,fun,enthusiasm,happiness,
-1 => anger,sadness,hate,worry,boredom